# Alphalens + Quantopian | How To

For more information on how to read and understand the plots look at:

- [Example notebook](https://github.com/quantopian/alphalens/blob/master/alphalens/examples/tear_sheet_walk_through.ipynb) from the repo.
- [Alphalens Docs](http://quantopian.github.io/alphalens) for an analysis of a professional alpha factor.

In [ ]:
import numpy as np
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, AverageDollarVolume
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.filters import Q500US

In [ ]:
MORNINGSTAR_SECTOR_CODES = {
     -1: 'Misc',
    101: 'Basic Materials',
    102: 'Consumer Cyclical',
    103: 'Financial Services',
    104: 'Real Estate',
    205: 'Consumer Defensive',
    206: 'Healthcare',
    207: 'Utilities',
    308: 'Communication Services',
    309: 'Energy',
    310: 'Industrials',
    311: 'Technology' ,    
}

## Define our factor

In [ ]:
class LowVol(CustomFactor):
    inputs = [Returns(window_length=2)]
    window_length = 25
    
    def compute(self, today, assets, out, close):
        out[:] = -np.nanstd(close, axis=0)

# Define our universe

In [ ]:
universe = Q500US()

## Create the Pipeline

In [ ]:
pipe_low_vol = Pipeline(
    columns={
        'LowVol' : LowVol(mask=universe),
        'Sector': Sector(mask=universe),
    },
    screen=universe
)

## Run the Pipeline

In [ ]:
results = run_pipeline(pipe_low_vol, '2015-06-30', '2016-06-30')

In [ ]:
results.head()

## Get pricing

In [ ]:
assets = results.index.levels[1].unique()
# We need to get a little more pricing data than the 
# length of our factor so we can compare forward returns.
# We'll tack on another month in this example.
pricing = get_pricing(assets, start_date='2015-06-30', end_date='2016-07-31', fields='open_price')

In [ ]:
pricing.head()

## Use Alphalens to create a factor tear sheet

In [ ]:
import alphalens

factor_data = alphalens.utils.get_clean_factor_and_forward_returns(results['LowVol'],
                                                                   pricing,
                                                                   quantiles=5,
                                                                   groupby=results['Sector'],
                                                                   periods=(1,5,10))

alphalens.tears.create_full_tear_sheet(factor_data)